In [1]:
%pip install langchain-core langgraph>0.2.27
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_api_key
from langchain_groq import ChatGroq
# Initialize the ChatGroq model with a specific model
model = ChatGroq(model="llama3-8b-8192")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.021666667, 'prompt_time': 0.002161398, 'queue_time': 0.012543162, 'total_time': 0.023828065}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f71e1d7-592f-44fe-9ac4-bd9544cb9d2a-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information or recall previous conversations. Each time you interact with me, it's a new conversation and I don't retain any information about you. So, I don't actually know your name. Would you like to introduce yourself?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 15, 'total_tokens': 86, 'completion_time': 0.059166667, 'prompt_time': 0.000147809, 'queue_time': 0.013832220999999999, 'total_time': 0.059314476}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7882aa04-079f-412e-8395-4ef31b453f3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 71, 'total_tokens': 86})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 40, 'total_tokens': 46, 'completion_time': 0.005, 'prompt_time': 0.005011337, 'queue_time': 0.018947663, 'total_time': 0.010011337}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b67bccc-514f-4473-81ac-9e5c7a7d5808-0', usage_metadata={'input_tokens': 40, 'output_tokens': 6, 'total_tokens': 46})

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

In [9]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [10]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [11]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize, but I don't have any information about your name. I'm a large language model, I don't have the ability to know or remember personal information about individuals, including their names. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to tell me your name, I'd be happy to learn it and use it in our conversation!


In [12]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is still Bob!


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, shiver me timbers! 'Tis a pleasure to make yer acquaintance, Jim me lad! Welcome aboard me ship o' knowledge, where we'll be answerin' yer questions like a barnacle on a sunken treasure chest! What be yer first query, matey?


In [16]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, ye be askin' yerself, eh? Well, matey, I be tellin' ye that yer name be Jim, as ye yerself just said! Ye be Jim, the scurvy dog! Now, what be yer next question, matey?


In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [18]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hola Bob! ¡Bienvenido! ¿En qué puedo ayudarte hoy? (Hello Bob! Welcome! How can I help you today?)


In [20]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Tu nombre es Bob! (Your name is Bob!)


In [21]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

c:\Users\nvish\anaconda3\envs\AIPro\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [22]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me earlier that your name is Bob!


In [24]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me what 2 + 2 is!


In [25]:
output

{'messages': [SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}, id='33658866-f354-4d4c-9d9f-3a26f7cdcfd0'),
  HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='f41394c5-1c12-417d-acc5-6e5114e72982'),
  AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='846a98dd-8406-42fd-bb34-d66de5090b78'),
  HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}, id='1d1a8c88-881c-4a53-931e-5315eefa5457'),
  AIMessage(content='nice', additional_kwargs={}, response_metadata={}, id='4e12b60b-5e02-42f7-ad68-1da33edc297d'),
  HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}, id='b5af88cb-4131-4fe0-8b6a-fb67d3efe1c7'),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}, id='5ba667ba-4570-4a66-b728-4b3fcb1fdd0e'),
  HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}, id='3f1460ad-b21a-49ac-ab41-0f9339116

In [28]:
output["messages"][-1].content

'You asked me what 2 + 2 is!'